In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/My\ Drive/data/*.zip .
!unzip /content/sudoku.zip
!mkdir /content/test
!unzip /content/sudoku_test.zip -d /content/test
!mv /content/test/sudoku.csv /content/sudoku_test.csv

In [ ]:
!git clone https://github.com/cloughurd/drl-sudoku.git
!mv drl-sudoku/data/* .
!mv drl-sudoku/models/* .

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
# from tqdm import tqdm
import random
from torch.utils.data.sampler import SubsetRandomSampler


from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

In [2]:
from cnn import BasicNet
from dataloader import *
from display import *

train_loader = get_dataloader(root='./', batch_size=2, train=True, cap_train=3000000)
test_loader = get_dataloader(root='./', batch_size=512, train=False)
const_x, const_y = next(iter(test_loader))
print(const_x, '\n', const_y)
net = BasicNet()
# net = net.cuda()
optimizer = optim.Adam(net.parameters())
objective = nn.CrossEntropyLoss()
losses = []

NameError: name 'get_dataloader' is not defined

In [ ]:
def train(num_epochs=20, print_val_freq=1000):
    step = 0
    for i in range(num_epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            
            x = x.cuda().float()
            y_hat = net(x)
            
            loss = objective(y_hat.transpose(1,2), y.reshape((-1, 81)).long())
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            
            if step % print_val_freq == 0:
                const_y_hat = net(const_x.cuda().float())
                print_tensor_puzzle(const_y_hat[0].reshape(9,9,9))